In [2]:
import pandas as pd
import requests
import json
import ast

In [6]:
credits = pd.read_csv('Desktop/592-final-project/data/credits.csv')
#credits.head()

example_crew = credits['crew'][0]
#example_crew
crews = ast.literal_eval(example_crew)
#crews
director = [crew.get('id') for crew in crews if crew.get('job', '') == 'Director']
director = director[0] if director else -1
#director
def get_directors_id(crews):
    crews = ast.literal_eval(crews)
    director = [crew.get('id') for crew in crews if crew.get('job', '') == 'Director']
    director = director[0] if director else -1
    return director

credits['director_id'] = credits['crew'].apply(get_directors_id)

credits['director_id'] = credits['director_id'].astype('int')
#credits.head()

example_cast = credits['cast'][0]
#example_cast

casts = ast.literal_eval(example_cast)
#casts

cast_order = [cast.get('id') for cast in casts if cast.get('order', '') <= 1]
#cast_order = cast_order[:2] if cast else -1
#cast_order = cast_order0[:2] if cast else -1
#cast_order

def get_director_id(crews):
    crews = ast.literal_eval(crews)
    director = [crew.get('id') for crew in crews if crew.get('job', '') == 'Director']
    return director[0] if director else -1

def get_cast_ids(casts):
    casts = ast.literal_eval(casts)
    top2 = [cast.get('id') for cast in casts if cast.get('order', 9999) < 2]
    primary = top2[0] if top2 else -1
    secondary = top2[1] if len(top2) >= 2 else -1
    return pd.Series([primary, secondary])

credits['director_id'] = credits['crew'].apply(get_director_id)
credits[['primary_cast','secondary_cast']] = credits['cast'].apply(get_cast_ids)
credits.head()
len(credits)

45476

In [7]:
credits.head()

,cast,crew,id,director_id,primary_cast,secondary_cast
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,7879,31,12898
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,4945,2157,8537
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,26502,6837,3151
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,2178,8851,9780
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,56106,67773,3092


In [8]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   cast            45476 non-null  object
 1   crew            45476 non-null  object
 2   id              45476 non-null  int64 
 3   director_id     45476 non-null  int64 
 4   primary_cast    45476 non-null  int64 
 5   secondary_cast  45476 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 2.1+ MB


In [5]:
director_id_set = set(credits['director_id'].to_list())
primary_cast_id_set = set(credits['primary_cast'].to_list())
secondary_cast_id_set = set(credits['secondary_cast'].to_list())


POPULARITIES

In [9]:
popularities = pd.read_csv('Desktop/592-final-project/data/popularities.csv')

In [10]:
popularities=popularities[['id','gender','popularity']]

In [11]:
popularities.head()

,id,gender,popularity
0,1,2,5.670
1,2,2,9.630
2,3,2,22.432
3,4,1,6.699
4,5,2,6.512


In [12]:
len(popularities)
popularities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46359 entries, 0 to 46358
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          46359 non-null  int64  
 1   gender      46359 non-null  int64  
 2   popularity  46359 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 1.1 MB


Merging dataframe popularities and credits by director id

In [13]:
mergedf1=pd.merge(credits, popularities, left_on='director_id', right_on='id',how="left")
mergedf1

,cast,crew,id_x,director_id,primary_cast,secondary_cast,id_y,gender,popularity
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,7879,31,12898,7879.0,2.0,4.430
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,4945,2157,8537,4945.0,2.0,4.032
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,26502,6837,3151,26502.0,2.0,1.083
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,2178,8851,9780,2178.0,2.0,3.893
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,56106,67773,3092,56106.0,2.0,1.056
...,...,...,...,...,...,...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050,1182809,240240,-1,1182809.0,0.0,0.600
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109,1051381,1043186,111636,1051381.0,2.0,0.980
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758,67753,23764,2059,67753.0,2.0,1.260
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506,1085341,544742,1090923,1085341.0,2.0,0.600


In [14]:
mergedf1.rename(columns={'gender': 'director_gender', 'popularity': 'director_popularity'}, inplace=True)

In [15]:
mergedf1.drop(['id_y'], axis=1)

,cast,crew,id_x,director_id,primary_cast,secondary_cast,director_gender,director_popularity
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,7879,31,12898,2.0,4.430
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,4945,2157,8537,2.0,4.032
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,26502,6837,3151,2.0,1.083
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,2178,8851,9780,2.0,3.893
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,56106,67773,3092,2.0,1.056
...,...,...,...,...,...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050,1182809,240240,-1,0.0,0.600
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109,1051381,1043186,111636,2.0,0.980
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758,67753,23764,2059,2.0,1.260
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506,1085341,544742,1090923,2.0,0.600


In [16]:
mergedf1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45476 entries, 0 to 45475
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   cast                 45476 non-null  object 
 1   crew                 45476 non-null  object 
 2   id_x                 45476 non-null  int64  
 3   director_id          45476 non-null  int64  
 4   primary_cast         45476 non-null  int64  
 5   secondary_cast       45476 non-null  int64  
 6   id_y                 44589 non-null  float64
 7   director_gender      44589 non-null  float64
 8   director_popularity  44589 non-null  float64
dtypes: float64(3), int64(4), object(2)
memory usage: 3.5+ MB


In [17]:
mergedf2_primarycast=pd.merge(mergedf1, popularities, left_on='primary_cast', right_on='id',how="left")

In [18]:
mergedf2_primarycast=mergedf2_primarycast.drop(['id_y','id'], axis=1)
mergedf2_primarycast.rename(columns={'id_x': 'movie_id', 'gender': 'primary_cast_gender','popularity':'primary_cast_popularity'}, inplace=True)
mergedf2_primarycast

,cast,crew,movie_id,director_id,primary_cast,secondary_cast,director_gender,director_popularity,primary_cast_gender,primary_cast_popularity
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,7879,31,12898,2.0,4.430,2.0,18.883
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,4945,2157,8537,2.0,4.032,2.0,10.278
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,26502,6837,3151,2.0,1.083,2.0,5.380
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,2178,8851,9780,2.0,3.893,1.0,2.608
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,56106,67773,3092,2.0,1.056,2.0,3.292
...,...,...,...,...,...,...,...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050,1182809,240240,-1,0.0,0.600,1.0,1.380
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109,1051381,1043186,111636,2.0,0.980,1.0,1.400
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758,67753,23764,2059,2.0,1.260,1.0,6.376
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506,1085341,544742,1090923,2.0,0.600,2.0,0.980


In [19]:
#merge on Seconday cast id
merged3_secondarycast=pd.merge(mergedf2_primarycast,popularities,left_on='secondary_cast', right_on='id',how="left")

In [20]:
merged3_secondarycast=merged3_secondarycast.drop(['id'],axis=1)
merged3_secondarycast.rename(columns={ 'gender': 'secondary_cast_gender','popularity':'secondary_cast_popularity'}, inplace=True)
merged3_secondarycast.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45476 entries, 0 to 45475
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   cast                       45476 non-null  object 
 1   crew                       45476 non-null  object 
 2   movie_id                   45476 non-null  int64  
 3   director_id                45476 non-null  int64  
 4   primary_cast               45476 non-null  int64  
 5   secondary_cast             45476 non-null  int64  
 6   director_gender            44589 non-null  float64
 7   director_popularity        44589 non-null  float64
 8   primary_cast_gender        42777 non-null  float64
 9   primary_cast_popularity    42777 non-null  float64
 10  secondary_cast_gender      35544 non-null  float64
 11  secondary_cast_popularity  35544 non-null  float64
dtypes: float64(6), int64(4), object(2)
memory usage: 4.5+ MB


Data cleaning -Drop NA

In [21]:
# drop rows with NA

In [22]:
merged3_secondarycast=merged3_secondarycast.dropna(how='any')

In [23]:
merged3_secondarycast.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35327 entries, 0 to 45474
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   cast                       35327 non-null  object 
 1   crew                       35327 non-null  object 
 2   movie_id                   35327 non-null  int64  
 3   director_id                35327 non-null  int64  
 4   primary_cast               35327 non-null  int64  
 5   secondary_cast             35327 non-null  int64  
 6   director_gender            35327 non-null  float64
 7   director_popularity        35327 non-null  float64
 8   primary_cast_gender        35327 non-null  float64
 9   primary_cast_popularity    35327 non-null  float64
 10  secondary_cast_gender      35327 non-null  float64
 11  secondary_cast_popularity  35327 non-null  float64
dtypes: float64(6), int64(4), object(2)
memory usage: 3.5+ MB


In [24]:
merged3_secondarycast.head()

,cast,crew,movie_id,director_id,primary_cast,secondary_cast,director_gender,director_popularity,primary_cast_gender,primary_cast_popularity,secondary_cast_gender,secondary_cast_popularity
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,7879,31,12898,2.0,4.430,2.0,18.883,2.0,5.016
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,4945,2157,8537,2.0,4.032,2.0,10.278,2.0,1.370
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,26502,6837,3151,2.0,1.083,2.0,5.380,2.0,4.775
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,2178,8851,9780,2.0,3.893,1.0,2.608,1.0,4.735
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,56106,67773,3092,2.0,1.056,2.0,3.292,1.0,7.819


In [25]:
# convert data type of gender columns into integer
merged3_secondarycast.director_gender = merged3_secondarycast.director_gender.astype(int)
merged3_secondarycast.primary_cast_gender = merged3_secondarycast.primary_cast_gender.astype(int)
merged3_secondarycast.secondary_cast_gender = merged3_secondarycast.secondary_cast_gender.astype(int)

In [26]:
merged3_secondarycast.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35327 entries, 0 to 45474
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   cast                       35327 non-null  object 
 1   crew                       35327 non-null  object 
 2   movie_id                   35327 non-null  int64  
 3   director_id                35327 non-null  int64  
 4   primary_cast               35327 non-null  int64  
 5   secondary_cast             35327 non-null  int64  
 6   director_gender            35327 non-null  int32  
 7   director_popularity        35327 non-null  float64
 8   primary_cast_gender        35327 non-null  int32  
 9   primary_cast_popularity    35327 non-null  float64
 10  secondary_cast_gender      35327 non-null  int32  
 11  secondary_cast_popularity  35327 non-null  float64
dtypes: float64(3), int32(3), int64(4), object(2)
memory usage: 3.1+ MB


In [27]:
# find max min of column values
merged3_secondarycast.max(axis=0)

cast                         [{'cast_id': 99, 'character': 'Wade Wilson / D...
crew                         [{'credit_id': '59e7cd319251414ab80005e0', 'de...
movie_id                                                                463906
director_id                                                            1907940
primary_cast                                                           1902699
secondary_cast                                                         1906016
director_gender                                                              3
director_popularity                                                     56.706
primary_cast_gender                                                          3
primary_cast_popularity                                                 56.706
secondary_cast_gender                                                        3
secondary_cast_popularity                                               56.706
dtype: object

In [28]:
merged3_secondarycast.min(axis=0) 

cast                         [{'cast_id': 0, 'character': "Billy 'The Great...
crew                         [{'credit_id': '52fe420dc3a36847f800001f', 'de...
movie_id                                                                     2
director_id                                                                  1
primary_cast                                                                 1
secondary_cast                                                               2
director_gender                                                             -1
director_popularity                                                         -1
primary_cast_gender                                                         -1
primary_cast_popularity                                                     -1
secondary_cast_gender                                                       -1
secondary_cast_popularity                                                   -1
dtype: object

In [30]:
count_directorgender = merged3_secondarycast['director_gender'].value_counts()
print(count_directorgender)
# 0 not specified
# 1female
# 2male
# 3 non-binary

 2    28352
 0     4972
 1     1837
-1      164
 3        2
Name: director_gender, dtype: int64


In [31]:
count_primarycastgender = merged3_secondarycast['primary_cast_gender'].value_counts()

In [32]:
print(count_primarycastgender)

 2    23317
 1     9365
 0     2532
-1       99
 3       14
Name: primary_cast_gender, dtype: int64


In [33]:
count_primarycastgender = merged3_secondarycast['secondary_cast_gender'].value_counts()
print(count_primarycastgender)

 2    17945
 1    13652
 0     3574
-1      148
 3        8
Name: secondary_cast_gender, dtype: int64


In [34]:
#remove outliers using zscore for popularities
#from scipy import stats
#import numpy as np
#merged3_secondarycast['z_score_director_popularity']=(merged3_secondarycast.director_popularity-merged3_secondarycast.director_popularity.mean())/merged3_secondarycast.director_popularity.std(ddof=0)
##lst=[]
#for i in merged3_secondarycast.loc[merged3_secondarycast['z_score_director_popularity'].abs()>=3].director_popularity:
   # lst.append(i)

#len(lst)

In [35]:
#director_popularity: for populatirity minimu value is -1 
# popularity has no upperbond, it is better to just retain the original data
len(merged3_secondarycast[merged3_secondarycast['director_popularity']==-1])
#drop rows if value is -1

merged3_secondarycast = merged3_secondarycast[merged3_secondarycast.director_popularity != -1] # 156 values are -1 for column directory_popularity
#primary_cast_popularity
merged3_secondarycast=merged3_secondarycast = merged3_secondarycast[merged3_secondarycast.primary_cast_popularity != -1]
#secondary_cast_popularity
merged3_secondarycast=merged3_secondarycast = merged3_secondarycast[merged3_secondarycast.secondary_cast_popularity != -1]
merged3_secondarycast.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34933 entries, 0 to 45474
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   cast                       34933 non-null  object 
 1   crew                       34933 non-null  object 
 2   movie_id                   34933 non-null  int64  
 3   director_id                34933 non-null  int64  
 4   primary_cast               34933 non-null  int64  
 5   secondary_cast             34933 non-null  int64  
 6   director_gender            34933 non-null  int32  
 7   director_popularity        34933 non-null  float64
 8   primary_cast_gender        34933 non-null  int32  
 9   primary_cast_popularity    34933 non-null  float64
 10  secondary_cast_gender      34933 non-null  int32  
 11  secondary_cast_popularity  34933 non-null  float64
dtypes: float64(3), int32(3), int64(4), object(2)
memory usage: 3.1+ MB


Merge on movie id for merged3_secondarycast and movies_metadata.csv on Kaggle

In [38]:
movies_metadata=pd.read_csv("Desktop/592-final-project/data/movies_metadata.csv")

movies_metadata= movies_metadata[movies_metadata.id != '1997-08-20']
movies_metadata= movies_metadata[movies_metadata.id != '2012-09-29']
movies_metadata= movies_metadata[movies_metadata.id != '2014-01-01']


movies_metadata['id']=movies_metadata.id.astype(int)
movies_metadata.head()

C:\Users\olivia\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [39]:
df=pd.merge(merged3_secondarycast,movies_metadata,left_on='movie_id',right_on='id',how="left")

In [40]:
df[['movie_id','id']]

,movie_id,id
0,862,862
1,8844,8844
2,15602,15602
3,31357,31357
4,11862,11862
...,...,...
34976,222848,222848
34977,30840,30840
34978,111109,111109
34979,67758,67758


In [41]:
df=df.drop(['id'],axis=1)

In [42]:
df

,cast,crew,movie_id,director_id,primary_cast,secondary_cast,director_gender,director_popularity,primary_cast_gender,primary_cast_popularity,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,7879,31,12898,2,4.430,2,18.883,...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,4945,2157,8537,2,4.032,2,10.278,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,26502,6837,3151,2,1.083,2,5.380,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,2178,8851,9780,2,3.893,1,2.608,...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,56106,67773,3092,2,1.056,2,3.292,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34976,"[{'cast_id': 1, 'character': 'Kira (as Cassand...","[{'credit_id': '5757f36ac3a3687d6f000e8a', 'de...",222848,42634,9313,929095,2,0.608,1,3.545,...,1995-01-01,0.0,85.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Caged Heat 3000,False,3.5,1.0
34977,"[{'cast_id': 1, 'character': 'Sir Robert Hode'...","[{'credit_id': '52fe44439251416c9100a899', 'de...",30840,17784,29459,139,2,0.802,2,1.521,...,1991-05-13,0.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Robin Hood,False,5.7,26.0
34978,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109,1051381,1043186,111636,2,0.980,1,1.400,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
34979,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758,67753,23764,2059,2,1.260,1,6.376,...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0


In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34981 entries, 0 to 34980
Data columns (total 35 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   cast                       34981 non-null  object 
 1   crew                       34981 non-null  object 
 2   movie_id                   34981 non-null  int64  
 3   director_id                34981 non-null  int64  
 4   primary_cast               34981 non-null  int64  
 5   secondary_cast             34981 non-null  int64  
 6   director_gender            34981 non-null  int32  
 7   director_popularity        34981 non-null  float64
 8   primary_cast_gender        34981 non-null  int32  
 9   primary_cast_popularity    34981 non-null  float64
 10  secondary_cast_gender      34981 non-null  int32  
 11  secondary_cast_popularity  34981 non-null  float64
 12  adult                      34981 non-null  object 
 13  belongs_to_collection      3991 non-null   obj

In [46]:
df.to_csv('Desktop/592-final-project/data/merged_datasets.csv')